In [1]:
import numpy as np
import scipy as sp
from scipy.interpolate import griddata, RegularGridInterpolator

from astropy.io import fits

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = fits.open('delta_transmission_RMplate.fits')
len(data)

1247

## Getting Coordinates

In [4]:
ra_mean = 213.704
dec_mean = 53.083

In [5]:
def get_matrix(data):

    ra_list, dec_list, z_list = [np.nan], [np.nan], [np.nan]

    for i in range(1, len(data)):
        header = data[i].header
        ra_list.append(header['RA'])
        dec_list.append(header['DEC'])

    ra, dec = np.deg2rad(np.array(ra_list)), np.deg2rad(np.array(dec_list))
    mean_ra, mean_dec = np.nanmean(ra), np.nanmean(dec)

    k_par = np.array([np.sin(mean_dec) * np.cos(mean_ra), np.sin(mean_dec) * np.sin(mean_ra), np.cos(mean_dec)])
    k_perp = np.array([-np.sin(mean_ra), np.cos(mean_ra), 0])

    K = np.array([
        [0, -k_perp[2], k_perp[1]],
        [k_perp[2], 0, -k_perp[0]],
        [-k_perp[1], k_perp[0], 0]
    ])

    R = np.eye(3) + np.sin(mean_dec) * K + (1 - np.cos(mean_dec)) * np.matmul(K, K)
    Q = np.eye(3) - np.sin(mean_dec) * K + (1 - np.cos(mean_dec)) * np.matmul(K, K)

    return Q

In [6]:
def read_line(line):
    ra = (line.header['RA']) * np.pi / 180
    dec = (line.header['DEC']) * np.pi / 180
    rcomov = line.data['RCOMOV']
    value = line.data['DELTA_T']

    x = np.sin(dec) * np.cos(ra) * rcomov
    y = np.sin(dec) * np.sin(ra) * rcomov
    z = np.cos(dec) * rcomov
    v = value

    return x, y, z, v

In [7]:
def get_array(data):

    x_list, y_list, z_list, v_list = [], [], [], []
    for i in range(1, len(data)):
        x, y, z, v = read_line(data[i])
        x_list.append(x)
        y_list.append(y)
        z_list.append(z)
        v_list.append(v)

    # concatenating together
    x = np.hstack(x_list)
    y = np.hstack(y_list)
    z = np.hstack(z_list)
    v = np.hstack(v_list)

    # rotating the points
    x, y, z = np.dot(Q, np.array([x, y, z]))

    # shifting cordinates
    x -= x.min()
    y -= y.min()
    z -= z.min()

    return x, y, z, v

In [8]:
Q = get_matrix(data)
Q

array([[ 0.72393142, -0.18421634,  0.66482151],
       [-0.18421634,  0.87707526,  0.44362522],
       [-0.66482151, -0.44362522,  0.60100667]])

In [9]:
x, y, z, v = get_array(data)
x.shape, y.shape, z.shape, v.shape

((2433528,), (2433528,), (2433528,), (2433528,))

In [10]:
x.min(), x.max()

(0.0, 268.99086363983884)

In [11]:
y.min(), y.max()

(0.0, 297.841541444388)

In [12]:
z.min(), z.max()

(0.0, 1160.0498469214522)

In [13]:
xi = np.arange(0, 300, 1)
yi = np.arange(0, 300, 1)
zi = np.arange(0, 300, 1)

In [14]:
xx, yy, zz = np.meshgrid(xi, yi, zi)

In [16]:
xx.shape, yy.shape, zz.shape

((300, 300, 300), (300, 300, 300), (300, 300, 300))

In [ ]:
vv = griddata((x,y,z),v,(xx,yy,zz),method='linear')

In [ ]:
np.save('vvl.npy', vv)